In [8]:
uploaded = files.upload()


Saving resume_csv.csv to resume_csv (4).csv


In [9]:

import os
print(os.listdir())  # List all files in the current directory


['.config', 'resume_csv.csv', 'resume_csv (2).csv', 'resume_csv (4).csv', 'resume_csv (1).csv', 'resume_csv (3).csv', 'sample_data']


In [12]:
import pandas as pd
from google.colab import files

# Upload the file
print("Upload the resume dataset (CSV format):")
uploaded = files.upload()

# Get the correct file name dynamically
file_name = list(uploaded.keys())[0]
print(f"File successfully uploaded: {file_name}")

# Load the dataset
df = pd.read_csv('resume_csv (4).csv')

# Print column names to verify
print("Column names in the dataset:", df.columns.tolist())


Upload the resume dataset (CSV format):


Saving resume_csv.csv to resume_csv (7).csv
File successfully uploaded: resume_csv (7).csv
Column names in the dataset: ['address', 'career_objective', 'skills', 'educational_institution_name', 'degree_names', 'passing_years', 'educational_results', 'result_types', 'major_field_of_studies', 'professional_company_names', 'company_urls', 'start_dates', 'end_dates', 'related_skils_in_job', 'positions', 'locations', 'responsibilities', 'extra_curricular_activity_types', 'extra_curricular_organization_names', 'extra_curricular_organization_links', 'role_positions', 'languages', 'proficiency_levels', 'certification_providers', 'certification_skills', 'online_links', 'issue_dates', 'expiry_dates', '\ufeffjob_position_name', 'educationaL_requirements', 'experiencere_requirement', 'age_requirement', 'responsibilities.1', 'skills_required', 'matched_score']


In [13]:
required_columns = ["skills", "responsibilities", "job_position_name"]
missing_cols = [col for col in required_columns if col not in df.columns]

if missing_cols:
    print("Missing columns:", missing_cols)
else:
    print("All required columns are present!")


Missing columns: ['job_position_name']


In [15]:
df = df.rename(columns={"\ufeffjob_position_name": "job_position_name"})


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine resume content for ranking
df["resume_text"] = df["skills"].fillna('') + " " + df["responsibilities"].fillna('')

# Define function to rank candidates
def rank_candidates(job_description, resumes):
    documents = [job_description] + resumes
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)

    job_desc_vector = tfidf_matrix[0:1]  # Job description vector
    resume_vectors = tfidf_matrix[1:]    # Resume vectors

    similarity_scores = cosine_similarity(job_desc_vector, resume_vectors).flatten()
    ranked_candidates = sorted(zip(df["job_position_name"], similarity_scores), key=lambda x: x[1], reverse=True)

    return ranked_candidates

# Get Job Description Input
job_description = input("Enter the job description: ")

# Rank Candidates
ranked_results = rank_candidates(job_description, df["resume_text"].tolist())

# Display Top 3 Results
top_3_df = pd.DataFrame(ranked_results[:3], columns=["Job Position", "Similarity Score"])
print("\nTop 3 Ranked Candidates based on Job Description:\n")
print(top_3_df)


Enter the job description: engineer

Top 3 Ranked Candidates based on Job Description:

                   Job Position  Similarity Score
0  Executive/ Sr. Executive -IT          0.273723
1      Senior Software Engineer          0.273470
2           Mechanical Engineer          0.271879
